In [1]:
import os

os.chdir('..')

In [2]:
import ast
import json
import pandas as pd
from tqdm import tqdm

from utils.read_data import read_jsonl
from RE.scripts.settings import MAP_LABELS
from RE.RelationExtractor import RelationExtractor

tqdm.pandas()

/opt/conda/envs/spacy_gpu/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Read data

In [3]:
data_path = './RE/assets/annotations_test.jsonl'

In [4]:
data = read_jsonl(data_path)

### Get predictions

In [5]:
data['ents'] = data.spans.apply(lambda x: [(x[i]['start'], x[i]['end'], x[i]['label']) for i in range(len(x))])

In [6]:
re = RelationExtractor(model='trf')

In [8]:
data['preds'] = data.progress_apply(lambda x: re.get_predictions(x.text, 0.4, True, x.ents, False)[1], axis=1)

100%|██████████| 57/57 [00:38<00:00,  1.48it/s]


In [9]:
data.relations.iloc[0]

[{'head': 8,
  'child': 18,
  'head_span': {'start': 36,
   'end': 50,
   'token_start': 5,
   'token_end': 8,
   'label': 'TISSUE'},
  'child_span': {'start': 93,
   'end': 121,
   'token_start': 15,
   'token_end': 18,
   'label': 'TARGET'},
  'label': 'activ'},
 {'head': 24,
  'child': 18,
  'head_span': {'start': 142,
   'end': 168,
   'token_start': 22,
   'token_end': 24,
   'label': 'TARGET'},
  'child_span': {'start': 93,
   'end': 121,
   'token_start': 15,
   'token_end': 18,
   'label': 'TARGET'},
  'label': 'activ'},
 {'head': 63,
  'child': 24,
  'head_span': {'start': 326,
   'end': 345,
   'token_start': 59,
   'token_end': 63,
   'label': 'CHEMICAL'},
  'child_span': {'start': 142,
   'end': 168,
   'token_start': 22,
   'token_end': 24,
   'label': 'TARGET'},
  'label': 'activ'}]

In [21]:
data['truth'] = data.apply(lambda x: [(x.text[x.relations[i]['head_span']['start']: x.relations[i]['head_span']['end']], 
                                       x.text[x.relations[i]['child_span']['start']: x.relations[i]['child_span']['end']],
                                       MAP_LABELS[x.relations[i]['label']], 
                                      1) for i in range(len(x.relations))], axis=1)

In [32]:
data.truth.iloc[4]

[('rnf169', 'h2a-lys13', 'Bind', 1), ('rnf169', 'lys15', 'Bind', 1)]

In [33]:
data.preds.iloc[4]

[[('rnf169', 'h2a-lys13', 'Bind', 0.97017896)],
 [('rnf169', 'lys15', 'Bind', 0.96965617)],
 [('rnf169', 'ubiquitin', 'Bind', 0.9320422)]]

In [31]:
data.text.iloc[3]

"when compared to controls, the density of 125i-ngf binding sites was reduced by 30% in the striatum of patients with progressive supranuclear palsy but not reduced in that of patients with parkinson's disease. 125i-ngf binding was not significantly decreased in the nucleus basalis of meynert in either diseases"